In [ ]:
import visa                             # Import PyVISA library
import time
from struct import *                    # Import Struct module (interpret strings as packed binary data)
import numpy as np
#import VISAresourceExtentions

In [ ]:
# Initialization

rm = visa.ResourceManager()             # Create Resource Manager object
rs = rm.list_resources()                # Method to list the available resources
print(rs[0])                            
counter = rm.open_resource(rs[0])       # Assign the returned object to the instrument variable (i.e. counter)

In [ ]:
# Initial settings

print(counter.query('*IDN?'))           # Query the Identification string 
counter.write('*RST;*CLS')              # Reset the instrument, clear the Error queue
counter.timeout = 1e7                   # Acquisition timeout (ms) - set it higher than the acquisition time
counter.query('SYST:ERR?')              # Error Checking 

In [ ]:
#Basic settings

counter.write("FUNC 'FREQ:BTB 1'")                                 # Setup for frequency back-to-back measurement from channel A (1) - Set up for period back-to-back is "FUNC 'PER:BTB 1'" 
counter.write('CALC:AVER:STAT OFF')                                # Enable/disable statitics; 
counter.write('INP:LEV:AUTO OFF; :INP:LEV 0')                      # Enable/Disable autotrigger on channel A; Trigger level (V)

counter.write('CAL:INT:AUTO OFF; :DISP:ENAB OFF')                  # Enable/Disable reciprocal counter that uses an interpolating technique to increase the resolution (if OFF -> Dead time = 0); Enable/Disable the display (if OFF increase the GPIB speed)
counter.write('FORMAT:TINF ON; :FORMAT PACKED')                    # Read timestamp of each measurement; Redout in ASCII/REAL/PACKED mode - Readout format: ASCII/REAL -> [freq (Hz), tstamp (sec)] ; PACKED -> [freq (Hz), tstamp (picosec)] 
counter.write('SENSE:ACQ:APER 4e-6')                               # Gate/Pacing time (sec), minimum: 4 microsec
counter.write('TRIG:COUNT 1; :ARM:COUNT 3e6')                      # Measure N samples inside one block (Triggerings); Number M of blocks (Armings) - Max(N*M) = 3.5e6 (Size of the memory) - with CNT-91 it can be set to INF (non-stop continuous measurements), with minimum Gate time = 50 microsec (recommended: 100 microsec)
counter.query('SYST:ERR?')                                         # Error Checking 
#time.sleep(1)                                                      # Wait (sec)

In [ ]:
# Perform  M ('ARM:COUNT') block measurements of 1 ('TRIG:COUNT') sample each (Zero Dead time between samples/blocks)
# Fetch the data of one acquisiton ( Total number of samples/measurements = M ) over multiple cycles 

##### If M < 3.5e6 samples, then the minimum Gate time = 4e-6s ('INIT:CONT OFF'), if you want to fetch M > 3.5e6 samples ('INIT:CONT ON'), then the minimum Gate time = 5e-5s (recommended: 1e-4s) #####

#counter.write('INIT:CONT ON')               # Initialize infinite continuous acquisition ( only if ':ARM:COUNT' is set to INF ) - Arm loop will continue endlessly for 107 days
#counter.query('INIT;*OPC?')                 # Initialize acquisition; *OPC? query waits until the acquisition ends

start = time.time()

for i in range(300):
    
    # Read Data in BINARY format (packed)
    
    #counter.write('FETCH:ARR? 1000')                                                              
    #counter.query_binary_values(datatype='s', is_big_endian=True)                                  
    bytesvalues = counter.query_binary_values('FETCH:ARR? 1e4', datatype='s', is_big_endian=True)   # Fetch Binary values - Rate = 7,313*10^3 Samples/s (Maximum fetchable samples = 1e4)
    freqtimelist = unpack('>'+'dQ'*int(1e4), bytesvalues[0])                                        # Convert (unpack) into a readable format - Readout format (tuple) -> (freq [Hz], timestamps [ps], freq, tstamp, ...)
    #freqtimelist = bytesvalues                                                                      # Save list of bytes
    #print(bytesvalues) 
    
    # Read Data in ASCII format
    
    #freqtimelist = counter.query_ascii_values('FETCH:ARR? 1e4', converter = 's')  # Fetch and Convert list of ASCII values into a list of strings (Maximum Fetchable samples = 1e4)
    
    #if  freqtimelist[-1] == '' :    del freqtimelist[-1]                          # Delete the last element of the list when it is an empty string = ''
    #freqtimelist[-1] = freqtimelist[-1].split('\n')[0]                            # Delete in the last element of the list of strings every character after the number ( i.e. \n0E..)
    #freqtimelist = [float(i) for i in freqtimelist]                               # Convert list of strings into float numbers
    print(freqtimelist)                                                         
    
    if (i==0): freqtimeList = freqtimelist                                        # Create a bigger List appending all the fetched data lists during the loop     
    else:  freqtimeList = freqtimeList + freqtimelist    
           
    end = time.time() 
    print(end-start)                                                              # Single acquisition time (sec)
    
#print(len(freqtimeList))                                                         # Number of elements of the Big List
                                                
end = time.time()
print(end-start)                                                                  # Total acquisition time (sec)
#counter.query('SYST:ERR?')                                                       # Error Checking 

In [ ]:
# Write binary string to file

BigBytesString = b''.join(freqtimeList)     # Join the all the bytes string elements of the list into one bytes string

f = open('/home/raffo/Scrivania/AllanDeviation/Counter_VCO/FreqBTB-Tstamps_Tau4e-6s_Samples3e6_VCO_Binary', 'w+b')
f.write(BigBytesString)
f.close()

In [ ]:
# Read binary string from file

f = open('/home/raffo/Scrivania/AllanDeviation/Counter_VCO/FreqBTB-Tstamps_Tau4e-6s_Samples3e6_VCO_Binary', 'r+b')
BigBytesString = f.read()

# Convert (unpack) binary string into a ASCII tuple

Bytes = len(BigBytesString)                                    # Number of bytes of the binary string
freqtimeList = unpack('>'+'dQ'*int(Bytes/16), BigBytesString)  # Bytes of the single (freq/tstamp) sample = 16 -> int(Bytes/16) is the total number of samples

In [ ]:
# Overlapping ADEV calculation

start = time.time()

freqtimearray = np.array(freqtimeList)                     # Convert list to numpy array
freqarray=(freqtimearray[::2])                             # Take the elements of the array at even steps
timearray=(freqtimearray[1::2])                            # Take the elements of the array at odd steps
 
StepSize = (timearray[2:]-timearray[1:-1]).mean()          # Calculate the mean time delay between two consecutive frequency samples
StepSize *= 1e-12                                          # From psec -> sec (only if data are taken in PAKED form)
#print(StepSize)

freqmean = freqarray.mean()                                # Calculate the mean frequency over all the frequency samples
#print(freqmean)
                                              
Adevlist = []                                              # Define list: list=[] ; Define numpy array:  arr=np.array([])
Sampleslist = []

Taulist = [1e-5 * 10**(n/5) for n in range(20)]            # Define list of gate times (sec) - NotEvenlySpaced: 1e-5 * 2n - EvenlySpaced: 1e-5 * 10**(n/5) - ...

for tau in Taulist:
    
    Tratio = int(round(tau/StepSize))                      # Take the nearest integer to the Tratio (i.e. Number of freq samples inside a specific Gate time)
    #print(Tratio)
        
    freqarr = freqarray[::Tratio]                                                                   # Define an array with number of elements = Tratio
    freqarr = np.array( [freqarray[Tratio*i:Tratio*(i+1)].mean() for i in range(len(freqarr))] )    # Average all the freq samples (# = Tratio) inside a specific Gate time
    #print(len(freqarr))
        
    print(f'Gate time = {tau} s, Samples = {len(freqarr)}') 
                
    deltafreqarray = (freqarr[1:]-freqarr[:-1])**2         # Calculate the square of all the differences of consecutive pairs of frequencies
    #print(len(deltafreqarray))
    adev = (np.mean(deltafreqarray)/2)**(1/2)              # Calculate Adev
    print(adev)
    #print(adev/freqmean)
        
    Adevlist.append(adev)                                                 # Append a value to the list at each iteration
    Sampleslist.append(len(freqarr))                                      # Creating the Samples array for each Gate time (append the number of samples value at each iteration)
         
    #end = time.time()     
    #print(end-start)                                                      # Acquisition time for each Gate time (sec)

end = time.time()
print(end-start)                                                          # Total acquisition time (sec)                                                             # Total acquisition time (sec)

In [ ]:
print(Taulist)  
print(Adevlist)
print(Sampleslist)

In [ ]:
# ADEV vs Gate time plot

import matplotlib.pyplot as plt
import pandas as pd

# Convert lists to numpy arrays

TauArray = np.array(Taulist)      
AdevArray = np.array(Adevlist)
SamplesArray = np.array(Sampleslist)

print(repr(SamplesArray))
print(repr(TauArray))
print(repr(AdevArray))

ADEVArray = AdevArray/freqmean   # Calculate normalized Adev

print(repr(ADEVArray))

plt.plot(TauArray, ADEVArray, 'o-')
plt.xscale('log')
plt.yscale('log')
#plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
plt.title('ADEV vs Gate time')
#plt.ylabel('Hz')
plt.xlabel('sec')
plt.show()

In [ ]:
counter.write('*RST;*CLS') 

In [ ]:
counter.close()